Shreeya Kokate

CWID: 20005256

Section: CS 583A Tuesday 9.30am-12pm



Question:  Neural  Machine  Translation  (10  Points)
You should train a neural network model to automatically translate from English words to their 
transformated forms. The transformation rules are as follows:

• if the first letter is a consonant, then that letter is moved to the end of the word and “ay” is appended, e.g., slow →lowsay

• if a word starts with a vowel, then append “way” at the end, e.g., amoeba →amoebaway.

• some consonant pairs like “sh” are moved together to end of the word with “ay” appended, e.g., shallow →allowshay.

To translate a sentence, simply translate each word independently.


ANSWER: This is a Pig-Latin Pattern

In [ ]:
import pandas as pd
import string
from collections import defaultdict
from sklearn.model_selection import train_test_split

import keras
import keras.backend as K
K.clear_session()
from keras.models import Model
from keras.layers import Input, SimpleRNN, Dense, LSTM
from keras.layers import Embedding
from keras.preprocessing import sequence
from tensorflow.keras.optimizers import Adam

In [ ]:
dataset = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data.txt', sep = ' ', header = None)
dataset.columns = ['SourceWords', 'TargetWords']
dataset['SourceWords'] = dataset['SourceWords'].astype(str).apply(lambda x:" ".join(list(x)))
dataset['TargetWords'] = dataset['TargetWords'].astype(str).apply(lambda x:" ".join(list(x)))
dataset.head()

In [ ]:
dataset.columns = ('EncoderSourceWords', 'TargetWords')
dataset['DecoderSourceWords'] = dataset['TargetWords'].apply(lambda x: '<begin> '+ x[:-1])
dataset.head()

In [ ]:
train_dataset, test_dataset = train_test_split(dataset, test_size = 0.2)

X_train_encoder = [[vocab[char] for char in x.split() if vocab[char] != 0] for x in train_dataset["EncoderSourceWords"].values]
X_test_encoder = [[vocab[char] for char in x.split() if vocab[char] != 0] for x in test_dataset["EncoderSourceWords"].values]

X_train_decoder = [[vocab[char] for char in x.split() if vocab[char] != 0] for x in train_dataset["DecoderSourceWords"].values]
X_test_decoder = [[vocab[char] for char in x.split() if vocab[char] != 0] for x in test_dataset["DecoderSourceWords"].values]

Y_train = [[vocab[char] for char in x.split() if vocab[char] !=0] for x in train_dataset["TargetWords"].values]
Y_test = [[vocab[char] for char in x.split() if vocab[char] !=0] for x in test_dataset["TargetWords"].values]

In [ ]:
X_train_encoder = sequence.pad_sequences(X_train_encoder, maxlen = 20)
X_test_encoder = sequence.pad_sequences(X_test_encoder, maxlen = 20)

X_train_decoder = sequence.pad_sequences(X_train_decoder, maxlen = 20)
X_test_decoder = sequence.pad_sequences(X_test_decoder, maxlen = 20)

Y_train = sequence.pad_sequences(Y_train, maxlen = 20)
Y_test = sequence.pad_sequences(Y_test, maxlen = 20)

In [ ]:
count = 1
vocab_list = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '-', '`', '<begin>']
vocab = defaultdict(lambda : 0)
for i in vocab_list:
    if (vocab[i] == 0): 
        vocab[i] = count
        count += 1

In [ ]:
def Encoder(vocab_size = 30 + 1, embedding_size = 10, input_len= 20):
    EncoderSource = Input(shape = (20,))
    a = Embedding(vocab_size, embedding_size)(EncoderSource)
    encoder = SimpleRNN(embedding_size, return_state = True)
        
    EncoderTarget, final_state = encoder(a)
    return EncoderSource, EncoderTarget, final_state

In [ ]:
def Decoder(final_state, vocab_size = 30 + 1, embedding_size = 10, input_len = 20):
    DecoderSource = Input(shape = (20,))
    a = Embedding(vocab_size, embedding_size)(DecoderSource)
    decoder = SimpleRNN(embedding_size, return_state = True, return_sequences = True)

    model_output, _ = decoder(a, initial_state = final_state)
    DecoderTarget = Dense(vocab_size, activation = 'softmax')(model_output)
    return DecoderSource, DecoderTarget

In [ ]:
EncoderSource, EncoderTarget, final_state  = Encoder()
DecoderSource, DecoderTarget = Decoder(final_state)
model = Model([EncoderSource, DecoderSource], DecoderTarget)
model.summary()

In [ ]:
model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy')

In [ ]:
model.fit([X_train_encoder, X_train_decoder], Y_train, batch_size = 16, epochs = 30, verbose = 2) 

In [ ]:
Y_Predict = model.predict([X_test_encoder, X_test_decoder])

metric = keras.metrics.SparseTopKCategoricalAccuracy(k=1)
metric.update_state(Y_test,Y_Predict)

print('Accuracy = {}'.format(metric.result().numpy()))

Q4. Question:  If you input a very long sentence,  do you have problem of translation this sentence? If so, what could be potentially the problem? Write down the answer. 

ANSWER: 

If we input a very long sentence, we will surely have problem of translation in this sentence.

Attempting to back-propagate across very long input sequences may result in vanishing gradients, and in turn, an unlearnable model. This may result in the problem of very long training times.

This may be happening due to activation function of the model.


Q5. What model or activation function can you use the improve long sentence translation?  Implement such a model. 

ANSWER: 

Long Short-Term Memory or LSTM recurrent neural networks are capable of learning and remembering over long sequences of inputs.

Afew techniques can be:
1. Truncate Sequences
2. Use Truncated Backpropagation Through Time
3. Sometimes, summarize the input sequences
4. Random Sampling
5. Use an Encoder-Decoder Architecture

In [ ]:
def Encoder(vocab_size = 30 + 1, embedding_size = 10, input_len= 20):
    EncoderSource = Input(shape = (20,))
    b = Embedding(vocab_size, embedding_size)(EncoderSource)
    encoder = LSTM(embedding_size, return_state=True)
        
    EncoderTarget, final_state_h, final_state_c = encoder(b)
    return EncoderSource, EncoderTarget, [final_state_h, final_state_c]

In [ ]:
def Decoder(final_state, vocab_size = 30 + 1, embedding_size = 10, input_len = 20):
    DecoderSource = Input(shape = (20,))
    a = Embedding(vocab_size, embedding_size)(DecoderSource)
    decoder = LSTM(embedding_size, return_state = True, return_sequences = True)

    model_output, _,_ = decoder(a, initial_state = final_state)
    DecoderTarget = Dense(vocab_size, activation = 'softmax')(model_output)
    return DecoderSource, DecoderTarget

In [ ]:
EncoderSource, EncoderTarget, final_state  = Encoder()
DecoderSource, DecoderTarget = Decoder(final_state)
model = Model([EncoderSource, DecoderSource], DecoderTarget)
model.summary()

In [ ]:
model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy')
model.fit([X_train_encoder, X_train_decoder], Y_train, batch_size = 16, epochs = 30, verbose = 2)

In [ ]:
Y_Predict = model.predict([X_test_encoder, X_test_decoder])

metric = keras.metrics.SparseTopKCategoricalAccuracy(k = 1)
metric.update_state(Y_test,Y_Predict)

print('Accuracy = {}'.format(metric.result().numpy()))